# Part 1 - Plotting neutron energy spectra in a histogram form

This example creates a simple sphere of water and tallies the neutrons in two different ways:

- Neutron flux averaged across the cell.
- Neutron current on the rear surface.

This section creates a simple material, geometry and settings. This model is used in both the neutron current tally and the neutron flux tally.

In [ ]:
import openmc

# MATERIALS

# Due to the hydrogen content water is a very good neutron moderator
my_material = openmc.Material(name='water')
my_material.add_element('H', 1, percent_type='ao')
my_material.add_element('O', 2, percent_type='ao')
my_material.set_density('g/cm3', 1)

mats = openmc.Materials([my_material])


# GEOMETRY

# surfaces
vessel_inner_surface = openmc.Sphere(r=500)
vessel_rear_surface = openmc.Sphere(r=530)
# Currently it is not possible to tally on boundary_type='vacuum' surfaces
outer_surface = openmc.Sphere(r=550, boundary_type='vacuum')

# cells
inner_vessel_cell = openmc.Cell(region=-vessel_inner_surface)
# inner_vessel_cell is filled with a void / vacuum by default

blanket_cell = openmc.Cell(region=-vessel_rear_surface & +vessel_inner_surface)
blanket_cell.fill = my_material

outer_vessel_cell = openmc.Cell(region=+vessel_rear_surface & -outer_surface)
# this is filled with a void / vacuum by default

universe = openmc.Universe(cells=[inner_vessel_cell,blanket_cell, outer_vessel_cell])
geom = openmc.Geometry(universe)


# SIMULATION SETTINGS

# Instantiate a Settings object
sett = openmc.Settings()
sett.batches = 100
sett.inactive = 0 # the default is 10, which would be wasted computing for us
sett.particles = 1000
sett.run_mode = 'fixed source'

# Create a DT point source
source = openmc.Source()
source.space = openmc.stats.Point((0, 0, 0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14e6], [1])
sett.source = source

This section section adds a tally for the average neutron flux across a cell.

In [ ]:
#creates an empty tally object
tallies = openmc.Tallies()

# sets up filters for the tallies
neutron_particle_filter = openmc.ParticleFilter(['neutron'])
energy_bins = openmc.mgxs.GROUP_STRUCTURES['CCFE-709']
energy_filter = openmc.EnergyFilter(energy_bins)


# setup the filters for the cell tally
cell_filter = openmc.CellFilter(blanket_cell) 

# create the tally
cell_spectra_tally = openmc.Tally(name='cell_spectra_tally')
cell_spectra_tally.scores = ['flux']
cell_spectra_tally.filters = [cell_filter, neutron_particle_filter, energy_filter]
tallies.append(cell_spectra_tally)

This section adds two surface current tallies - one on the inner sphere surface and one on the outer sphere surface.

In [ ]:
# sets up filters for the tallies
neutron_particle_filter = openmc.ParticleFilter(['neutron'])
energy_bins = openmc.mgxs.GROUP_STRUCTURES['CCFE-709']
energy_filter = openmc.EnergyFilter(energy_bins)

# setup the filters for the surface tally
front_surface_filter = openmc.SurfaceFilter(vessel_inner_surface)
back_surface_filter = openmc.SurfaceFilter(vessel_rear_surface)
# detects when particles across the surface

front_surface_spectra_tally = openmc.Tally(name='front_surface_spectra_tally')
front_surface_spectra_tally.scores = ['current']
front_surface_spectra_tally.filters = [front_surface_filter, neutron_particle_filter, energy_filter]
tallies.append(front_surface_spectra_tally)

back_surface_spectra_tally = openmc.Tally(name='back_surface_spectra_tally')
back_surface_spectra_tally.scores = ['current']
back_surface_spectra_tally.filters = [back_surface_filter, neutron_particle_filter, energy_filter]
tallies.append(back_surface_spectra_tally)

This section runs the simulation.

In [ ]:
# combine all the required parts to make a model
model = openmc.model.Model(geom, mats, sett, tallies)

# remove old files and runs OpenMC
!rm *.h5
results_filename = model.run()

This section extracts the cell tally data from the results file and plots neutron flux across the cell.

In [ ]:
from plotting_utils import create_plotly_figure, add_trace_to_figure

# open the results file
results = openmc.StatePoint(results_filename)

#extracts the tally values from the simulation results
cell_tally = results.get_tally(name='cell_spectra_tally')
cell_tally = cell_tally.get_pandas_dataframe()


fig = create_plotly_figure(y_axis_label='Neutrons per cm2 per source neutron')
add_trace_to_figure(
    figure=fig,
    energy_bins=energy_bins,
    values=cell_tally['mean'],
    std_dev=cell_tally['std. dev.']
)

This section extracts the surface tally data from the results file and plots neutron current through the inner and outer surfaces.

In [ ]:
from plotting_utils import create_plotly_figure

# open the results file
results = openmc.StatePoint(results_filename)

#extracts the tally values from the simulation results
back_surface_tally = results.get_tally(name='back_surface_spectra_tally')
back_surface_tally = back_surface_tally.get_pandas_dataframe()

front_surface_tally = results.get_tally(name='front_surface_spectra_tally')
front_surface_tally = front_surface_tally.get_pandas_dataframe()

fig = create_plotly_figure(y_axis_label='Neutrons per cm2 per source neutron')

add_trace_to_figure(
    figure=fig,
    energy_bins=energy_bins,
    values=back_surface_tally['mean'],
    std_dev=back_surface_tally['std. dev.'])

add_trace_to_figure(
    figure=fig,
    energy_bins=energy_bins,
    values=cell_tally['mean'],
    std_dev=cell_tally['std. dev.'])


#### Additional notes

- The current tally has a direction so particles can be +ve or -ve contributors.
- Sometimes an additional cellFromFilter is needed to detect the current in a certain direction.
- When comparing these results with other codes it might be necessary to scale them:
    - The OpenMC surface current needs to be divided by the surface area when comparing with MCNP.
    - The OpenMC cell flux needs to be divided by the cell volume when comparing with MCNP.


**Learning Outcomes for Part 1:**

- Neutron spectra can be tallied in OpenMC using flux or current tallies, and energy bins.
- Neutron spectra tend to be different on different surfaces and cells in the geometry.